In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.impute import SimpleImputer, KNNImputer
import numpy as np

# chargement jeu de donnée avec le moins de features
df = pd.read_csv('movies_clean_v2.csv')
df.head(20)
df.shape


(4544, 17)

In [4]:
# df_ml = df.drop(['acteurs', 'realisateur', 'studio'], axis=1)
# df_ml

,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,remake,titre,season,coeff_studio,scoring_acteurs,scoring_acteurs&realisateur
0,315000000,James Horner,2009-12-16,0,Franchise,Science Fiction,Etats-Unis,NaN,NaN,Avatar,Hiver,3,0.000000,0.275862
1,130000000,John Williams,2004-06-02,0,Franchise,Fantasy,Etats-Unis,NaN,NaN,Harry Potter et le prisonnier d'Azkaban,Été,3,0.724138,0.724138
2,200000000,John Williams,2017-12-13,0,Franchise,Fantasy,Etats-Unis,Kathleen Kennedy,NaN,Star Wars: Les derniers Jedi,Hiver,3,0.000000,0.000000
3,295000000,Alan Silvestri,2018-04-25,0,Franchise,Comicbook,Etats-Unis,Kevin Feige,NaN,Avengers: Infinity War,Printemps,3,0.689655,0.689655
4,350000000,NaN,2022-12-14,0,Franchise,Science Fiction,Etats-Unis,NaN,NaN,Avatar : la voie de l'eau,Hiver,3,0.448276,0.724138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4536,40000000,NaN,2011-04-06,0,NaN,Aventure - Action,Etats-Unis,NaN,Remake,Le Flingueur (2011),Printemps,0,0.034483,0.034483
4537,49515804,NaN,2006-10-04,0,NaN,Drame,Allemagne,NaN,NaN,Le Parfum : histoire d'un meurtrier,Automne,0,0.000000,0.000000
4538,27800000,NaN,2018-02-28,0,NaN,Comédie,France,NaN,NaN,La Ch’tite famille,Hiver,2,0.310345,0.517241
4539,2700000,NaN,2020-07-01,0,NaN,Comédie,France,NaN,NaN,Les Parfums,Été,0,0.000000,0.000000


In [10]:
# df['franchise_binary'] = df['franchise'].apply(lambda x: 1 if x == 'Franchise' else 0 if pd.isna(x) else np.nan)
# df['remake_binary'] = df['remake'].apply(lambda x: 1 if x == 'Remake' else 0 if pd.isna(x) else np.nan)




,acteurs,budget,compositeur,date,duree,entrees_premiere_semaine,genre,pays,producteur,realisateur,studio,titre,season,coeff_studio,scoring_acteurs&realisateur,franchise_binary,remake_binary
0,"Johnny Depp, Orlando Bloom, Geoffrey Rush, Kei...",225000000,Hans Zimmer,2006-08-02,150,2708112,Aventure - Action,Etats-Unis,Jerry Brukheimer,Gore Verbinski,Walt Disney Pictures,"Pirates des Caraïbes, le secret du coffre maudit",Été,3,0.310345,1,0
1,"Jean Reno, Christian Clavier",23000000,NaN,1998-02-11,118,2655916,Comédie,France,NaN,JeanMarie Poiré,GBVI,Les Visiteurs II - Les Couloirs du temps,Hiver,2,0.103448,1,0
2,"Daniel Radcliffe, Emma Watson, Ralph Fiennes, ...",150000000,John Williams,2005-11-30,153,3088796,Fantasy,Etats-Unis,NaN,Mike Newell,Warner Bros.,Harry Potter et la coupe de feu,Automne,3,0.758621,1,0
3,"Daniel Craig, Gad Elmaleh, Simon Pegg",135000000,John Williams,2011-10-26,107,3158313,Aventure - Action,Etats-Unis,Steven Spielberg,Steven Spielberg,Sony Pictures,Les Aventures de Tintin : Le secret de la Licorne,Automne,3,0.586207,0,0
4,"Robert Downey Jr, Scarlett Johansson, Chris He...",220000000,Alan Silvestri,2012-04-25,142,2041362,Comicbook,Etats-Unis,Kevin Feige,Joss Whedon,Walt Disney Pictures,Marvel's The Avengers,Printemps,3,0.482759,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4539,"Matt Dillon, Eva Green",6500000,NaN,2019-11-27,106,51977,Drame,France,NaN,NaN,Pathé,Proxima,Automne,2,0.000000,0,0
4540,Jim Carrey,72000000,NaN,2002-05-08,152,51858,Drame,Etats-Unis,NaN,Frank Darabont,Warner Bros.,The Majestic,Printemps,3,0.068966,0,0
4541,"Colin Farrell, Jennifer Connelly, Russell Crow...",60000000,Hans Zimmer,2014-03-12,129,51385,Romance,Etats-Unis,NaN,NaN,Warner Bros.,Un amour d'hiver,Printemps,3,0.655172,0,0
4542,Leïla Bekhti,3600000,NaN,2024-03-13,101,51632,Drame,France,NaN,NaN,Ad vitam,La Nouvelle femme,Printemps,2,0.068966,0,0


In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import pandas as pd
df = pd.read_csv('movies_clean_v3.csv')

df = df.drop(['acteurs', 'scoring_acteurs'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', RandomForestRegressor(n_estimators=100, random_state=0))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(mae)
print(r2)

141285.53370737075
0.5810602031557913


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
import pandas as pd
df = pd.read_csv('movies_clean_v3.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        # ('poly', PolynomialFeatures(degree=1, include_bias=False), numeric_features)  # Ajouter cette ligne pour les fonctionnalités polynomiales
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', CatBoostRegressor())])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae
r2 = r2_score(y_test, y_pred)
print(r2)
print(rmse)



Learning rate set to 0.050198
0:	learn: 415309.6729666	total: 12.9ms	remaining: 12.9s
1:	learn: 407273.4358949	total: 24.7ms	remaining: 12.3s
2:	learn: 398730.4552154	total: 35.5ms	remaining: 11.8s
3:	learn: 391955.9875126	total: 45.2ms	remaining: 11.3s
4:	learn: 385210.0539889	total: 55.3ms	remaining: 11s
5:	learn: 379194.7082572	total: 65.7ms	remaining: 10.9s
6:	learn: 373603.5964207	total: 74.7ms	remaining: 10.6s
7:	learn: 367797.1687815	total: 84.6ms	remaining: 10.5s
8:	learn: 362398.2171638	total: 94.1ms	remaining: 10.4s
9:	learn: 358237.3166352	total: 104ms	remaining: 10.3s
10:	learn: 353674.7103892	total: 114ms	remaining: 10.2s
11:	learn: 349820.9572467	total: 124ms	remaining: 10.2s
12:	learn: 345939.2238673	total: 134ms	remaining: 10.1s
13:	learn: 341941.3846526	total: 144ms	remaining: 10.1s
14:	learn: 338200.9127459	total: 153ms	remaining: 10.1s
15:	learn: 335200.6535141	total: 163ms	remaining: 10s
16:	learn: 331519.8350518	total: 172ms	remaining: 9.92s
17:	learn: 329126.88227

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import xgboost
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_absolute_error
import pandas as pd
df = pd.read_csv('movies_clean_v3.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', xgboost.XGBRegressor())])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

parameters = {'regressor__nthread':[4], #when use hyperthread, xgboost may become slower
              'regressor__objective':['reg:linear'],
              'regressor__learning_rate': [.03, 0.05, .07], #so called `eta` value
              'regressor__max_depth': [5, 6, 7],
              'regressor__min_child_weight': [4],
              'regressor__silent': [1],
              'regressor__subsample': [0.7],
              'regressor__colsample_bytree': [0.7],
              'regressor__n_estimators': [500]}


# Créer une instance de GridSearchCV
grid_search = GridSearchCV(model, parameters, cv=5, n_jobs=5, verbose=True)

# Effectuer la recherche sur la grille
grid_search.fit(X_train, y_train)

# Meilleurs hyperparamètres trouvés
best_params = grid_search.best_params_

# Meilleur score obtenu sur l'ensemble d'entraînement
best_score = grid_search.best_score_

# Meilleur modèle
best_model = grid_search.best_estimator_

# Utiliser le meilleur modèle pour prédire les étiquettes sur l'ensemble de test
y_pred = best_model.predict(X_test)

print("Meilleurs hyperparamètres:", best_params)
print("Meilleur score sur l'ensemble d'entraînement:", best_score)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:17:04] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:17:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:17:04] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:17:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used

Meilleurs hyperparamètres: {'regressor__colsample_bytree': 0.7, 'regressor__learning_rate': 0.03, 'regressor__max_depth': 5, 'regressor__min_child_weight': 4, 'regressor__n_estimators': 500, 'regressor__nthread': 4, 'regressor__objective': 'reg:linear', 'regressor__silent': 1, 'regressor__subsample': 0.7}
Meilleur score sur l'ensemble d'entraînement: 0.5371576377851727


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error
import pandas as pd
df = pd.read_csv('movies_clean_v3.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

# Gestion des valeurs manquantes pour les colonnes numériques et catégorielles
numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.drop('entrees_premiere_semaine')
numeric_transformer = SimpleImputer(strategy='median')

categorical_features = df.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Préparation du préprocesseur
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('poly', PolynomialFeatures(degree=2, include_bias=False), numeric_features)  # Ajouter cette ligne pour les fonctionnalités polynomiales
    ])

# Modèle de régression
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', Lasso(alpha=0.1))])

# Séparation des données
X = df.drop('entrees_premiere_semaine', axis=1)
y = df['entrees_premiere_semaine']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluation du modèle
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)
mae
r2 = r2_score(y_test, y_pred)
print(r2)
print(rmse)

0.23457010938188638
373367.7939145865


/home/utilisateur/Documents/cinema_prediction/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 222910783721.4056, tolerance: 65516167501.46724
  model = cd_fast.sparse_enet_coordinate_descent(


In [7]:
from pycaret.datasets import get_data
import pandas as pd 
df = df = pd.read_csv('movies_clean_v3.csv')

df = df.drop(['acteurs', 'scoring_acteurs', 'realisateur', 'studio'], axis=1)

df.to_csv('df_pycaret.csv')

dataset = get_data('df_pycaret')
#check the shape of data
dataset.shape


,Unnamed: 0,budget,compositeur,date,entrees_premiere_semaine,franchise,genre,pays,producteur,remake,titre,season,coeff_studio,scoring_acteurs&realisateur
0,0,315000000,James Horner,2009-12-16,2648596,Franchise,Science Fiction,Etats-Unis,NaN,NaN,Avatar,Hiver,3,0.275862
1,1,130000000,John Williams,2004-06-02,2503699,Franchise,Fantasy,Etats-Unis,NaN,NaN,Harry Potter et le prisonnier d'Azkaban,Été,3,0.724138
2,2,200000000,John Williams,2017-12-13,2510462,Franchise,Fantasy,Etats-Unis,Kathleen Kennedy,NaN,Star Wars: Les derniers Jedi,Hiver,3,0.000000
3,3,295000000,Alan Silvestri,2018-04-25,2565953,Franchise,Comicbook,Etats-Unis,Kevin Feige,NaN,Avengers: Infinity War,Printemps,3,0.689655
4,4,350000000,NaN,2022-12-14,2739848,Franchise,Science Fiction,Etats-Unis,NaN,NaN,Avatar : la voie de l'eau,Hiver,3,0.724138


(4541, 14)

In [8]:
from pycaret.regression import *

s = setup(data = dataset, target = 'entrees_premiere_semaine', session_id=123)

,Description,Value
0,Session id,123
1,Target,entrees_premiere_semaine
2,Target type,Regression
3,Original data shape,"(4541, 14)"
4,Transformed data shape,"(4541, 50)"
5,Transformed train set shape,"(3178, 50)"
6,Transformed test set shape,"(1363, 50)"
7,Numeric features,4
8,Categorical features,9
9,Rows with missing values,100.0%


In [11]:
best = compare_models()
best

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,12:43:10
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,234461.8800,149732496938.5104,384676.3356,0.1845,1.6120,17.2992,0.4460
knn,K Neighbors Regressor,224809.2574,150346439580.0738,386293.4335,0.1742,1.5050,13.4622,0.0870
catboost,CatBoost Regressor,238151.2590,157846148987.2522,395220.5592,0.1391,1.6170,17.4247,1.3980
gbr,Gradient Boosting Regressor,270184.7894,183072710177.3636,426076.4254,-0.0024,1.6968,19.0106,0.2420
rf,Random Forest Regressor,270091.1438,183169132272.3973,426176.5397,-0.0028,1.6968,18.9933,0.4680
dt,Decision Tree Regressor,270149.9334,183192797114.6479,426212.0873,-0.0030,1.6970,19.0013,0.0890
ridge,Ridge Regression,270485.2140,183412852633.9713,426466.0006,-0.0041,1.6977,19.0116,0.0780
lr,Linear Regression,270485.2140,183412852634.2998,426466.0006,-0.0041,1.6977,19.0116,0.3920
lasso,Lasso Regression,270484.9638,183412527193.8150,426465.6337,-0.0041,1.6977,19.0116,0.0800
en,Elastic Net,270485.1122,183412856345.3961,426465.9931,-0.0041,1.6977,19.0116,0.0780


ExtraTreesRegressor(n_jobs=-1, random_state=123)